In [2]:
%pwd

'/home/sesna/sesna/NLP Projects/Text-Summerizer-Project/research'

In [1]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/sesna/sesna/NLP Projects/Text-Summerizer-Project'

In [5]:
# Update entity - This function gives the return type of a function in config,yaml
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url: str
    local_data_file:Path
    unzip_dir:Path

In [6]:
from scr.textSummarizer.constants import *
from scr.textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):


                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)

                create_directories([self.config.artifacts_root])
                
    def get_data_ing(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])   

        data_ingestion_config = DataIngestionConfig(
               root_dir= config.root_dir,
               source_url= config.source_url,
               local_data_file=config.local_data_file,
               unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as rq
import zipfile
from textSummarizer.logging import logger
from scr.textSummarizer.utils.common import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = rq.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file alreday exists of size; {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        extracts the zip file into the data directory
        function returns None
        """
        unzip_path =  self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ing()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise

[2024-03-29 19:12:46,972: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-29 19:12:46,974: INFO: common: yaml file: /home/sesna/sesna/NLP Projects/Text-Summerizer-Project/params.yaml loaded successfully]
[2024-03-29 19:12:46,975: INFO: common: created directory at: artifacts]
[2024-03-29 19:12:46,975: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-29 19:12:46,976: INFO: 3854133850: file alreday exists of size; 7718 KB]
